# Phil's Fish Factory

In [1]:
import pandas as pd
import statsmodels.formula.api as sm

In [2]:
## bringing in dataset

loc1 = "Datasets/PhilsFishFactoryData.xlsx"
fish = pd.read_excel(loc1)
fish.head()

,Unnamed: 0,Day,Wholesale Price ($/lb),# Lobsters sold
0,NaN,Fri,7.0,4
1,NaN,sat,4.0,29
2,NaN,Sun,6.5,18
3,NaN,Mon,5.0,17
4,NaN,Tue,4.0,25


In [3]:
## drop column with null values

fish = fish.dropna(axis=1, how='all')
fish.head()

,Day,Wholesale Price ($/lb),# Lobsters sold
0,Fri,7.0,4
1,sat,4.0,29
2,Sun,6.5,18
3,Mon,5.0,17
4,Tue,4.0,25


In [4]:
## rename variables

fish['Price'] = fish['Wholesale Price ($/lb)']
fish['Lobs_Sold'] = fish['# Lobsters sold']
fish.head()

,Day,Wholesale Price ($/lb),# Lobsters sold,Price,Lobs_Sold
0,Fri,7.0,4,7.0,4
1,sat,4.0,29,4.0,29
2,Sun,6.5,18,6.5,18
3,Mon,5.0,17,5.0,17
4,Tue,4.0,25,4.0,25


In [5]:
## count differing values in 'Day' variable

pd.value_counts(fish['Day'])

Fri     2
Mon     2
Wed     2
Thur    2
Tue     2
Sun     2
Sat     1
sat     1
Name: Day, dtype: int64

In [6]:
## lowercase all values in 'Day' variable

fish['Day'] = fish['Day'].apply(lambda x: x.lower())

pd.value_counts(fish['Day'])

wed     2
fri     2
tue     2
thur    2
sat     2
mon     2
sun     2
Name: Day, dtype: int64

In [7]:
## create dummy variables for each day of the week

fishy = pd.get_dummies(fish['Day'])
fishy.head()

,fri,mon,sat,sun,thur,tue,wed
0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0


In [8]:
## create new dataset with renamed variables

newfish = fish[['Day','Price','Lobs_Sold']]
newfish.head()

,Day,Price,Lobs_Sold
0,fri,7.0,4
1,sat,4.0,29
2,sun,6.5,18
3,mon,5.0,17
4,tue,4.0,25


In [9]:
## join new dataset with dummy variables

newfish = newfish.join(fishy)
newfish.head()

,Day,Price,Lobs_Sold,fri,mon,sat,sun,thur,tue,wed
0,fri,7.0,4,1,0,0,0,0,0,0
1,sat,4.0,29,0,0,1,0,0,0,0
2,sun,6.5,18,0,0,0,1,0,0,0
3,mon,5.0,17,0,1,0,0,0,0,0
4,tue,4.0,25,0,0,0,0,0,1,0


In [10]:
## run correlation matrix 

newfish.corr()

,Price,Lobs_Sold,fri,mon,sat,sun,thur,tue,wed
Price,1.000000,-0.627493,0.507778,0.152333,-0.060933,0.223422,0.152333,-0.345289,-0.629644
Lobs_Sold,-0.627493,1.000000,-0.562715,-0.175645,0.211425,0.188656,-0.243952,0.484651,0.097581
fri,0.507778,-0.562715,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
mon,0.152333,-0.175645,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
sat,-0.060933,0.211425,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667
sun,0.223422,0.188656,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667
thur,0.152333,-0.243952,-0.166667,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667
tue,-0.345289,0.484651,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,1.000000,-0.166667
wed,-0.629644,0.097581,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,1.000000


The quantity of lobsters sold has a strong negative correlation to lobster price (r = -.63).

In [24]:
## Regression model predicting Number of Lobsters Sold; using Saturday as reference category for 'Day of the Week'

result1 = sm.ols(formula = 'Lobs_Sold ~ Price + sun + mon + tue + wed + thur + fri - 1', data=newfish).fit()
result1.summary()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Lobs_Sold   R-squared:                       0.851
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     5.723
Date:                Fri, 05 Oct 2018   Prob (F-statistic):             0.0174
Time:                        20:40:27   Log-Likelihood:                -47.490
No. Observations:                  14   AIC:                             109.0
Df Residuals:                       7   BIC:                             113.5
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Price          3.1747      1.345      2.361      0.050      -0.005       6.354
sun            1.4520     10.809      0.134      0.897     -24.107      27.011
mon           -5.7544     10.560     -0.545      0.603     -30.726      19.217
tue           14.3013      8.982      1.592      0.155      -6.938      35.540
wed            8.9760      8.247      1.088      0.312     -10.526      28.478
thur          -7.2544     10.560     -0.687      0.514     -32.226      17.717
fri          -18.2227     11.846     -1.538      0.168     -46.235       9.789
==============================================================================
Omnibus:                        0.934   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.627   Jarque-Bera (JB):                0.778
Skew:                           0.493   Prob(JB):                        0.678
Kurtosis:                       2.399   Cond. No.                         38.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
## Regression model predicting Number of Lobsters Sold; using Friday as reference category for 'Day of the Week'

result2 = sm.ols(formula = 'Lobs_Sold ~ Price + sat + sun + mon + tue + wed + thur - 1', data=newfish).fit()
result2.summary()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/scipy/stats/stats.py:1334: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Lobs_Sold   R-squared:                       0.909
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     9.966
Date:                Fri, 05 Oct 2018   Prob (F-statistic):            0.00355
Time:                        20:40:49   Log-Likelihood:                -44.065
No. Observations:                  14   AIC:                             102.1
Df Residuals:                       7   BIC:                             106.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Price          0.2637      0.776      0.340      0.744      -1.572       2.100
sat           19.6817      6.841      2.877      0.024       3.505      35.858
sun           18.9181      7.310      2.588      0.036       1.633      36.203
mon           10.9840      7.188      1.528      0.170      -6.012      27.980
tue           25.9454      6.432      4.034      0.005      10.736      41.155
wed           17.7090      6.095      2.905      0.023       3.296      32.122
thur           9.4840      7.188      1.320      0.229      -7.512      26.480
==============================================================================
Omnibus:                        0.810   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.667   Jarque-Bera (JB):                0.666
Skew:                          -0.156   Prob(JB):                        0.717
Kurtosis:                       1.979   Cond. No.                         28.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Regression Equation 

Number Lobsters Sold = (.26 x Wholesale Price) + (19.68 x ifSaturday) + (18.92 x ifSunday) + (10.98 x ifMonday) + (25.95 x ifTuesday) +(17.71 x ifWednesday) + (9.48 x ifThursday)

# Gleeful Consulting 

In [14]:
loc2 = "Datasets/GleeClubData.xlsx"
glee = pd.read_excel(loc2)
glee.head()

,team size,male/female ratio of team,enrollment in school,male/female ratio of School,population of district served by school,# of practices per month,# of competitions before regionals,Regionals Score
0,9,1.1,469,0.5,11432,1,3,3.160421
1,12,0.9,2028,1.1,64874,12,7,4.789190
2,23,1.4,1393,1.4,13843,10,3,3.676977
3,17,0.9,425,0.8,40959,9,0,3.641177
4,37,0.5,2111,1.0,26203,15,4,4.164019


In [15]:
## rename variables

glee['tm_size'] = glee['team size']
glee['mf_ratio_team'] = glee['male/female ratio of team']
glee['enrolled'] = glee['enrollment in school']
glee['mf_ratio_school'] = glee['male/female ratio of School']
glee['pop_district'] = glee['population of district served by school']
glee['prac_per_mnth'] = glee['# of practices per month']
glee['num_comps'] = glee['# of competitions before regionals']
glee['r_score'] = glee['Regionals Score']

glee.head()

,team size,male/female ratio of team,enrollment in school,male/female ratio of School,population of district served by school,# of practices per month,# of competitions before regionals,Regionals Score,tm_size,mf_ratio_team,enrolled,mf_ratio_school,pop_district,prac_per_mnth,num_comps,r_score
0,9,1.1,469,0.5,11432,1,3,3.160421,9,1.1,469,0.5,11432,1,3,3.160421
1,12,0.9,2028,1.1,64874,12,7,4.789190,12,0.9,2028,1.1,64874,12,7,4.789190
2,23,1.4,1393,1.4,13843,10,3,3.676977,23,1.4,1393,1.4,13843,10,3,3.676977
3,17,0.9,425,0.8,40959,9,0,3.641177,17,0.9,425,0.8,40959,9,0,3.641177
4,37,0.5,2111,1.0,26203,15,4,4.164019,37,0.5,2111,1.0,26203,15,4,4.164019


In [16]:
## create new dataset with renamed variables

newglee = glee[['tm_size','mf_ratio_team','enrolled','mf_ratio_school','pop_district','prac_per_mnth','num_comps','r_score']]
newglee.head()

,tm_size,mf_ratio_team,enrolled,mf_ratio_school,pop_district,prac_per_mnth,num_comps,r_score
0,9,1.1,469,0.5,11432,1,3,3.160421
1,12,0.9,2028,1.1,64874,12,7,4.789190
2,23,1.4,1393,1.4,13843,10,3,3.676977
3,17,0.9,425,0.8,40959,9,0,3.641177
4,37,0.5,2111,1.0,26203,15,4,4.164019


In [17]:
## run correlation matrix

newglee.corr()

,tm_size,mf_ratio_team,enrolled,mf_ratio_school,pop_district,prac_per_mnth,num_comps,r_score
tm_size,1.000000,0.039261,-0.002032,-0.027581,-0.027813,0.017570,-0.017728,0.037383
mf_ratio_team,0.039261,1.000000,0.014302,-0.022603,-0.043413,-0.039693,0.060830,-0.019025
enrolled,-0.002032,0.014302,1.000000,-0.007546,0.063731,-0.043782,-0.076423,0.031495
mf_ratio_school,-0.027581,-0.022603,-0.007546,1.000000,0.030516,0.002079,-0.048233,0.004502
pop_district,-0.027813,-0.043413,0.063731,0.030516,1.000000,0.018413,-0.083559,0.437973
prac_per_mnth,0.017570,-0.039693,-0.043782,0.002079,0.018413,1.000000,0.028975,0.634882
num_comps,-0.017728,0.060830,-0.076423,-0.048233,-0.083559,0.028975,1.000000,0.474863
r_score,0.037383,-0.019025,0.031495,0.004502,0.437973,0.634882,0.474863,1.000000


# Relevant Factors

The correlation matrix shows that the items most strongly correlated with Regional Score are: Population of District Served by School (r = .44), Number of Practices per Month (r = .63), and Number of Competitions before Regionals (r = .47).

In [18]:
## Running Linear Regression Model with Intercept (AdjR2 = .83)
model1 = sm.ols(formula = 'r_score ~ tm_size + mf_ratio_team + enrolled + mf_ratio_school + pop_district + prac_per_mnth + num_comps', data=newglee).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                r_score   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     213.1
Date:                Fri, 05 Oct 2018   Prob (F-statistic):          9.38e-111
Time:                        20:17:27   Log-Likelihood:                 145.71
No. Observations:                 300   AIC:                            -275.4
Df Residuals:                     292   BIC:                            -245.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.9498      0.055     53.518      0.000       2.841       3.058
tm_size             0.0026      0.001      2.078      0.039       0.000       0.005
mf_ratio_team      -0.0091      0.028     -0.322      0.748      -0.065       0.047
enrolled          3.99e-05   1.41e-05      2.834      0.005    1.22e-05    6.76e-05
mf_ratio_school     0.0181      0.029      0.632      0.528      -0.038       0.074
pop_district     4.285e-06    2.2e-07     19.502      0.000    3.85e-06    4.72e-06
prac_per_mnth       0.0379      0.001     25.842      0.000       0.035       0.041
num_comps           0.0613      0.003     21.046      0.000       0.056       0.067
==============================================================================
Omnibus:                       26.857   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              105.904
Skew:                           0.170   Prob(JB):                     1.01e-23
Kurtosis:                       5.891   Cond. No.                     4.48e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.48e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
## Running Linear Regression Model without Intercept (AdjR2 = .98)
model2 = sm.ols(formula = 'r_score ~ tm_size + mf_ratio_team + enrolled + mf_ratio_school + pop_district + prac_per_mnth + num_comps - 1', data=newglee).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                r_score   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     2632.
Date:                Fri, 05 Oct 2018   Prob (F-statistic):          2.56e-260
Time:                        20:17:30   Log-Likelihood:                -211.34
No. Observations:                 300   AIC:                             436.7
Df Residuals:                     293   BIC:                             462.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
tm_size             0.0309      0.004      8.341      0.000       0.024       0.038
mf_ratio_team       0.7463      0.081      9.263      0.000       0.588       0.905
enrolled            0.0003   4.38e-05      6.429      0.000       0.000       0.000
mf_ratio_school     0.8419      0.079     10.646      0.000       0.686       0.997
pop_district     6.846e-06   7.04e-07      9.728      0.000    5.46e-06    8.23e-06
prac_per_mnth       0.0588      0.005     12.673      0.000       0.050       0.068
num_comps           0.1007      0.009     10.893      0.000       0.083       0.119
==============================================================================
Omnibus:                        1.749   Durbin-Watson:                   1.878
Prob(Omnibus):                  0.417   Jarque-Bera (JB):                1.604
Skew:                          -0.073   Prob(JB):                        0.448
Kurtosis:                       2.673   Cond. No.                     2.13e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
## Running Linear Regression Model with highly correlated variables without Intercept (AdjR2 = .91)
model3 = sm.ols(formula = 'r_score ~ pop_district + prac_per_mnth + num_comps - 1', data=newglee).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                r_score   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     1057.
Date:                Fri, 05 Oct 2018   Prob (F-statistic):          4.17e-158
Time:                        20:19:12   Log-Likelihood:                -466.23
No. Observations:                 300   AIC:                             938.5
Df Residuals:                     297   BIC:                             949.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
pop_district   1.905e-05   1.43e-06     13.330      0.000    1.62e-05    2.19e-05
prac_per_mnth     0.1462      0.009     15.916      0.000       0.128       0.164
num_comps         0.2619      0.019     14.037      0.000       0.225       0.299
==============================================================================
Omnibus:                        6.387   Durbin-Watson:                   1.686
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.170
Skew:                          -0.311   Prob(JB):                       0.0457
Kurtosis:                       3.328   Cond. No.                     1.91e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Model 2 - Regression Equation 

Regionals Score = (0.03 x Team Size) + (0.75 x MF Ratio Team) + (0.0003 x Enrollment in School) + (0.84 x MF Ratio School) + (6.85 x District Population) + (0.06 x Practices per Month) + (0.10 x Competitions before Regionals)